# Cell Types Plotting

This notebook will build on our previous Cell Types notebook to help us (*finally!*) plot the data that we've pulled from the Allen Cell Types database.

### By the end of this notebook, you'll be able to:
* Create plots using `matplotlib.pyplot`
* Manipulate aspects of plots
* Create bar, box, and scatter plots from the Allen Cell Types metrics
* Plot raw recording traces from the Allen electrophysiology data

### Table of Contents
1. [Step One: Plotting tools](#one)
2. [Step Two: Get data](#two)
3. [Step Three: Plot our ephys metrics](#three)
4. [Step Four: Organize the raw traces](#four)
5. [Step Five: Plot the raw traces](#five)

<a id="one"></a>

## Step One: Get comfortable with our plotting tools

First, let's get set up for plotting by importing the necessary tool boxes.

In [ ]:
# Tell Jupyter to plot our plots inline
%matplotlib inline

# Import matplotlib and "pyplot" module
# plt is the common abbreviation for matplotlib's pyplot module
import matplotlib as mpl
import matplotlib.pyplot as plt

First, let's create a <a href="https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.random.html">random line</a> using our favorite scientific computing toolbox, and show how we can use the `matplotlib.pyplot` module to plot it.

Useful functions:
* `plt.plot()` create a plot from a list, array, pandas series, etc.
* `plt.show()` show the plot (not strictly necessary in Jupyter, necessary in other IDEs)
* `plt.xlabel()` and `plt.ylabel()` change x and y labels
* `plt.title()` add a title

In [ ]:
import numpy as np

# Generate a random line from 1 to 100 with 100 values
random_line = np.random.randint(1,100,100)
random_line

random_line_2 = random_line + 100

plt.scatter(random_line,random_line_2)
# plt.plot(random_line)
# plt.plot(random_line_2)
plt.show()

The `plt.hist()` function works really similarly (see documentation <a href="https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html">here</a>).

<div class="alert alert-success"><b>Task</b>: In the cell below:
    
1. Generate a random list of 100 data points from a standard normal distribution (Hint: Use <code>np.random.standard_normal()</code>, documentation <a href="https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.standard_normal.html#numpy.random.standard_normal">here</a>).
    
2. Plot a histogram of our the data. 
</div>

We can also set up multiple subplots on the same figure using `subplots`. This also creates separate **axes** (really, separate plots) which we can access and manipulate, particularly if you are plotting multiple lines. It's common to use the `subplots` command for easier access to axis attributes.

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(15,5))
ax[0,0].plot(random_line)
ax[0,0].set_ylabel('random values')

plt.ylabel('random values')
plt.show()

There are *many, many* different aspects of a figure that you could manipulate (and spend a lot of time manipulating). 

Style guides help with this a bit, they set a few good defaults. Below, we are setting figure parameters, and choosing a figure style (see all styles <a href="https://matplotlib.org/gallery/style_sheets/style_sheets_reference.html">here</a>, or how to create your own style <a href="https://matplotlib.org/tutorials/introductory/customizing.html">here</a>.)

You can test how these parameters change our plots by going back and re-plotting the plots above.

In [ ]:
# Set the figure "dots per inch" to be higher than the default (optional, based on your personal preference)
mpl.rcParams['figure.dpi'] = 100

# (Optional) Choose a figure style
print(plt.style.available)
plt.style.use('bmh')

<a id="two"></a>

## Step Two: Get metadata & electrophysiology data

Here, we'll condense the steps from the previous notebook into one cell.

<b>Note</b>: In order to run the line below, you need to have the AllenSDK installed. You can find information on how to do that <a href="http://alleninstitute.github.io/AllenSDK/install.html">here</a>. If you're running this on the UCSD Datahub, the Allen SDK has already been installed for you.

In [ ]:
from allensdk.core.cell_types_cache import CellTypesCache
from allensdk.api.queries.cell_types_api import CellTypesApi

import pandas as pd

# We'll then initialize the cache as 'ctc' (cell types cache)
ctc = CellTypesCache(manifest_file='cell_types/manifest.json')

human_df = pd.DataFrame(ctc.get_cells(species=[CellTypesApi.HUMAN])).set_index('id')
ephys_features = pd.DataFrame(ctc.get_ephys_features()).set_index('specimen_id')
human_ephys_df = human_df.join(ephys_features)
human_ephys_df.head()

<a id="three"></a>

## Step Three: Plot our ephys metrics

Our plotting goal for today is to compare spiny and aspiny cells in humans. We have two options: we could split the dataframe into aspiny and spiny (as you did in our CellTypes notebook), or use our plotting tools to plot the data separately.

Usefully, Pandas has some <a href="https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html">built-in plotting tools</a> to interact with Matplotlib, so we can actually just tell it to plot based on spiny and aspiny.

First, let's plot the **number of cells** that we have for each of the dendrite types. Recall that we can access a column by using the bracket notation, with the column name in parentheses in the bracket.

`value_counts()` is a method that will count up the number of instances of each value.
`plot()` is a Pandas method that will plot, depending on the `kind` argument you give it.

In [ ]:
# For the different values in dendrite_type column, get the value_counts, and plot as a bar plot.
human_ephys_df['dendrite_type'].value_counts().plot(kind='bar')

# Add y label
plt.ylabel('Number of cells')

# Show the plot!
plt.show()

Our dataframe contains a *lot* of different metrics on these cells. Let's remind ourselves what we have available by accessing the `columns` attribute.

In [ ]:
human_ephys_df.columns

Let's choose one of these columns and plot a boxplot. We'll do this with a call to pyplot ([examples here](https://matplotlib.org/gallery/pyplots/boxplot_demo_pyplot.html#sphx-glr-gallery-pyplots-boxplot-demo-pyplot-py)).

**Note**: This is actually *slightly* easier by using the methods of our dataframe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.boxplot.html), but it's a little bit more difficult to work with separate objects of the plot afterwards. Knowing how to create plots with a call to `plt.boxplots()` is a more universal way to work with various types of data structures, including dataframes, arrays, lists, etc.

<div class="alert alert-success"><b>Task</b>: Create a boxplot that compares spiny, aspiny, and sparsely spiny with the following steps:
    
1. Create a figure and axes by using <code>plt.subplots()</code>.
2. Save three different dataframes from your <code>human_ephys_df</code> by filtering for spiny, aspiny, and sparsely spiny.
3. Assign the 'fast_trough_v_long_square' of each of your spiny, aspiny, and sparsely spiny dataframes to three different pandas series objects (like a dataframe, but only one dimension). For example,
    <code>spiny_ft = spiny_data['fast_trough_v_long_square']</code>
4. Create a list of your three different pandas series, and assign it to <code>data</code>.
5. Create a boxplot by using <code>ax.boxplot()</code> and don't forget to show it!
6. Once you're sure the boxplot is working, add a few lines of code to change the xticks, as well as add x and y labels (<a href="https://matplotlib.org/api/axes_api.html#axis-limits">see documentation here</a>).</div>

In [ ]:
# Your plotting code here!


### Plot this data as a scatterplot

The built-in scatterplot methods in Pandas are bit clunky, so we'll use `plt.scatter()` instead ([documentation here](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.scatter.html#matplotlib.pyplot.scatter).

The syntax for a scatter plot is slightly longer if we want to label multiple groups with different colors. We'll actually loop through groups in order to create our plot:

In [ ]:
# Get possible dendrite types
dendrite_types = human_ephys_df['dendrite_type'].unique()

fig = plt.figure()

for d_type in dendrite_types:
    
    df = human_ephys_df[human_ephys_df['dendrite_type'] == d_type]
    
    plt.scatter(df['fast_trough_v_long_square'], 
                df['upstroke_downstroke_ratio_long_square'],
                label=d_type)
    
plt.ylabel("upstroke-downstroke ratio")
plt.xlabel("fast trough depth (mV)")
plt.legend(loc='best') 
    
plt.show()

<a id="four"></a>

## Step Four: Organize the raw recording traces

The Allen database also shares the raw data in addition to the computed metrics that we plotted above. Let's take a look at a few recording traces from cells with different upstroke and downstroke ratios, to help us get an understanding of what that metric is actually capturing.

<div class="alert alert-success"><b>Task</b>: Use the <code>sort_values()</code> method on <code>human_ephys_df</code> dataframe to sort it by 'upstroke_downstroke_ratio_long_square'. We also need to tell this method to set <code>ascending</code> to False, because the default value is True <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html">see documentation here</a>).

Assign your new dataframe to `df_upstroke_sorted`. Show the first five rows and make sure that it's sorted with the highest 'upstroke_downstroke_ratio_long_square' value at the top.</div>

Now, let's get the id and 'upstroke_downstroke_ratio_long_square' value for the first cell in our dataset. Remember that we set it up so that the index of our dataframe contains the ID numbers.

<div class="alert alert-success"><b>Task:</b> Above the <code>print()</code> statements below, do the following:
    
1. Assign the entire first row of the dataframe to `top_cell`.
2. Assign the index (id number) of the top cell to `cell_id`.
    
</div>

In [ ]:
# Your code here

print('Cell with biggest upstroke:downstroke ratio')
print('Specimen ID: ' + str(cell_id))
print('With upstroke-downstroke ratio: ' + str(top_cell['upstroke_downstroke_ratio_long_square']))

Now, we can use yet another method of our CellTypesCache instance (`ctc`) to get the raw electrophysiology data for this cell.

<div class="alert alert-success"><b>Task</b>: Use the <code>get_ephys_data()</code> method to get the raw data for our top cell. You'll need to give this method the `cell_id`. <a href="http://alleninstitute.github.io/AllenSDK/allensdk.core.cell_types_cache.html">Full documentation here</a> and <a href="https://alleninstitute.github.io/AllenSDK/cell_types.html">examples here</a>.</div>

This data will return us a NwbDataSet object. This stands for 'Neurodata Without Borders' and is an increasingly common file type that folks use to share data [(More information here)](https://www.nwb.org/). NWBdata files contain their own methods and data, including one called `get_sweep_numbers()` which will print a list of ID numbers for the "sweeps," or different recordings that were done with different levels of current. It also has a method `get_sweep_metadata` to extract the information about what stimulus was given on each sweep.

In the code block below, we'll extract the sweep numbers for when the stimulus amplitude was greater than 100.

In [ ]:
sweepIDs = upstroke_data.get_sweep_numbers()

sweepNum = len(sweepIDs)
for i in range(1,sweepNum):
    sweepMeta = upstroke_data.get_sweep_metadata(sweepIDs[i])
    if sweepMeta['aibs_stimulus_name'] == 'Long Square':
        if sweepMeta['aibs_stimulus_amplitude_pa'] > 100:
            print(sweepIDs[i])

We still need to clean up a few things before plotting. First, we'll use the `get_sweep()` method to get the actual raw data for one particular sweep. Then, we'll get the corresponding amplitude of the stimulus, voltage of the recording trace, and time of the recording.

In [ ]:
# choose a sweep number to plot
upstroke_sweep_id = 48
upstroke_sweep = upstroke_data.get_sweep(upstroke_sweep_id) 

i = upstroke_sweep["stimulus"] # in A
v = upstroke_sweep["response"] # in V
i *= 1e12 # this converts the value to pA
v *= 1e12 # this converts the value to mV

sampling_rate = upstroke_sweep["sampling_rate"] # in Hz
t = (np.arange(0, len(v)) * (1.0 / sampling_rate))

<a id="five"></a>

## Step Five: Plot the raw recording traces

*Finally*, let's plot our data.

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)

# axes 0 is our first plot, of the recorded voltage data
axes[0].plot(t, v, color='black')
axes[0].set_ylabel("mV")
axes[0].set_title("whole-cell patch recording")

#axes 1 is our second plot, of the stimulus trace
axes[1].plot(t, i, color='gray')
axes[1].set_ylabel("pA")
axes[1].set_xlabel("seconds")
axes[1].set_title("stimulus")

plt.show()

Hmm, it'd be nice to see the action potentials a little more clearly.

<div class="alert alert-success"><b>Task</b>: Modify the plotting script (hint, use the <code>xlim()</code> method of <code>plt</code>) so that you can actually see the shape of the action potential.</div>

Thankfully, now that we have the code above, we can use this to plot *traces for any stimulus and any cell in the dataset*.

<div class="alert alert-success"><b>Task</b>: Instead of plotting the <b>top</b> upstroke_downstroke_ratio cell, plot the bottom cell. Create as many cells as you need below in order to do this.</div>